In [ ]:
import cv2
import matplotlib
import numpy as np
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

image_gray = cv2.imread("sar_3.jpg", cv2.IMREAD_GRAYSCALE)


![Изображение](Figure_1.png)

In [ ]:
import copy
import math

# 1. Для изображения sar_3.jpg найти наиболее протяжённый участок
# (выделить линии при помощи преобразования Хафа)
bin_img = copy.deepcopy(image_gray)
threshold_value = 80
bin_img[image_gray < threshold_value] = 0
bin_img[image_gray >= threshold_value] = 255
canny = cv2.Canny(bin_img, 0, 250, apertureSize=3)
lines = cv2.HoughLines(canny, 1, np.pi / 180, 110)

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
        cv2.line(image_gray, pt1, pt2, (255, 0, 0), 2, cv2.LINE_AA)

plt.figure(figsize=(6, 6))
plt.title("Преобразование Хафа — выделение линий")
plt.imshow(cv2.cvtColor(image_gray, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

Самая длинная линия: (np.int32(2), np.int32(3), np.int32(223), np.int32(224)), длина = 312.54 px

![Изображение](Figure_2.png)

In [ ]:
# 2. Для изображения sar_3.jpg провести исследование алгоритмов бинаризации,
# выделить участок дорожной полосы

img = cv2.imread("sar_3.jpg", cv2.IMREAD_GRAYSCALE)

# Простая бинаризация 
T = 80
bin_simple = np.zeros_like(img)
bin_simple[img < T] = 0
bin_simple[img >= T] = 255

# Бинаризация Отсу 
_, bin_otsu = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Адаптивная бинаризация
bin_adapt = cv2.adaptiveThreshold(img, 255,
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY, 71, 21)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Точечная бинаризация")
plt.imshow(bin_simple, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Бинаризация Отсу")
plt.imshow(bin_otsu, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Адаптивная бинаризация")
plt.imshow(bin_adapt, cmap="gray")
plt.axis("off")

plt.suptitle("Сравнение методов бинаризации для sar_3.jpg", fontsize=14)
plt.tight_layout()
plt.show()


![Изображение](Figure_3.png)
Лучший результат показывает адаптивная бинаризация

In [ ]:
scale = 1
delta = 0
ddepth = cv2.CV_16S

grad_x = cv2.Sobel(img, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
grad_y = cv2.Sobel(img, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)

abs_grad_x = cv2.convertScaleAbs(grad_x)
abs_grad_y = cv2.convertScaleAbs(grad_y)

grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Градиент X (Собель)")
plt.imshow(abs_grad_x, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Градиент Y (Собель)")
plt.imshow(abs_grad_y, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Общий градиент")
plt.imshow(grad, cmap="gray")
plt.axis("off")

plt.suptitle("Оператор Собеля — градиенты по осям", fontsize=14)
plt.tight_layout()
plt.show()

![Изображение](Figure_4.png)